In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix,f1_score
from sklearn.preprocessing import StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/HR_comma_sep.csv")
df.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


In [35]:
# Liệt kê số lượng giá trị khác nhau trong mỗi cột
for col in df.columns:
    unique_count = df[col].nunique()  # Đếm số lượng giá trị duy nhất trong cột
    print(f"Cột '{col}' có {unique_count} giá trị khác nhau.")

Cột 'satisfaction_level' có 92 giá trị khác nhau.
Cột 'last_evaluation' có 65 giá trị khác nhau.
Cột 'number_project' có 6 giá trị khác nhau.
Cột 'average_montly_hours' có 215 giá trị khác nhau.
Cột 'time_spend_company' có 8 giá trị khác nhau.
Cột 'Work_accident' có 2 giá trị khác nhau.
Cột 'left' có 2 giá trị khác nhau.
Cột 'promotion_last_5years' có 2 giá trị khác nhau.
Cột 'sales' có 10 giá trị khác nhau.
Cột 'salary' có 3 giá trị khác nhau.


In [36]:
# Liệt kê các giá trị khác nhau của mỗi cột
unique_values = {col: df[col].unique() for col in df.columns}
print("Số cột trong data Student Score:   ",df.columns)

# Hiển thị kết quả
for col, values in unique_values.items():
    print(f"Cột '{col}' có các giá trị khác nhau: {values}")

Số cột trong data Student Score:    Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident', 'left',
       'promotion_last_5years', 'sales', 'salary'],
      dtype='object')
Cột 'satisfaction_level' có các giá trị khác nhau: [0.38 0.8  0.11 0.72 0.37 0.41 0.1  0.92 0.89 0.42 0.45 0.84 0.36 0.78
 0.76 0.09 0.46 0.4  0.82 0.87 0.57 0.43 0.13 0.44 0.39 0.85 0.81 0.9
 0.74 0.79 0.17 0.24 0.91 0.71 0.86 0.14 0.75 0.7  0.31 0.73 0.83 0.32
 0.54 0.27 0.77 0.88 0.48 0.19 0.6  0.12 0.61 0.33 0.56 0.47 0.28 0.55
 0.53 0.59 0.66 0.25 0.34 0.58 0.51 0.35 0.64 0.5  0.23 0.15 0.49 0.3
 0.63 0.21 0.62 0.29 0.2  0.16 0.65 0.68 0.67 0.22 0.26 0.99 0.98 1.
 0.52 0.93 0.97 0.69 0.94 0.96 0.18 0.95]
Cột 'last_evaluation' có các giá trị khác nhau: [0.53 0.86 0.88 0.87 0.52 0.5  0.77 0.85 1.   0.54 0.81 0.92 0.55 0.56
 0.47 0.99 0.51 0.89 0.83 0.95 0.57 0.49 0.46 0.62 0.94 0.48 0.8  0.74
 0.7  0.78 0.91 0.93 0.98 0.97 0.79 0.59 

In [37]:
df['sales'].value_counts()

,count
sales,
sales,4140
technical,2720
support,2229
IT,1227
product_mng,902
marketing,858
RandD,787
accounting,767
hr,739


In [38]:
df.isnull().sum() # kiểm tra giá trị null trong df

,0
satisfaction_level,0
last_evaluation,0
number_project,0
average_montly_hours,0
time_spend_company,0
Work_accident,0
left,0
promotion_last_5years,0
sales,0
salary,0


In [39]:
df['left'].value_counts() # kiểm tra số lượng nhân viên đã nghỉ việc và chưa nghỉ việc

,count
left,
0,11428
1,3571


In [40]:
df.describe()


,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   sales                  14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [42]:
target="left"
x=df.drop(labels=target,axis=1)
y=df[target]
print(f"X shape: {x.shape} Y shape: {y.shape}")

X shape: (14999, 9) Y shape: (14999,)


In [43]:
X_train,X_test,y_train,y_test=train_test_split(x,y,random_state=42,test_size=0.2)
print(f"X train shape : {X_train.shape} Y train shape: {y_train.shape}")
print(f"X test shape : {X_test.shape} Y test shape: {y_test.shape}")

X train shape : (11999, 9) Y train shape: (11999,)
X test shape : (3000, 9) Y test shape: (3000,)


In [44]:
print(X_train.columns)

Index(['satisfaction_level', 'last_evaluation', 'number_project',
       'average_montly_hours', 'time_spend_company', 'Work_accident',
       'promotion_last_5years', 'sales', 'salary'],
      dtype='object')


In [45]:
df['salary'].unique()

array(['low', 'medium', 'high'], dtype=object)

In [46]:
df['sales'].unique()

array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)

In [47]:
salary=['low','medium','high']
sales=df['sales'].unique()
ord_transformer=Pipeline(steps=[
    # ("imputer",SimpleImputer(strategy="most_frequent")),
    ("encoder",OrdinalEncoder(categories=[salary,sales]))
])

In [48]:
num_transformer=Pipeline(steps=[
    # ("imputer",SimpleImputer(strategy="mean")),
    ("scaler",StandardScaler())
])
result=num_transformer.fit_transform(X_train[['average_montly_hours']])
# Lặp qua các giá trị ban đầu và giá trị sau khi xử lý và in chúng
for i, j in zip(X_train[['average_montly_hours']].values, result):
    print(f"Before: {i[0]} After: {j[0]}")

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
Before: 272 After: 1.4215773399090235
Before: 219 After: 0.3600223326838756
Before: 246 After: 0.9008145061759322
Before: 242 After: 0.8206971471400719
Before: 131 After: -1.4025595661050492
Before: 271 After: 1.4015480001500584
Before: 222 After: 0.4201103519607708
Before: 199 After: -0.04056446249542547
Before: 207 After: 0.11967025557629497
Before: 207 After: 0.11967025557629497
Before: 255 After: 1.0810785640066176
Before: 186 After: -0.3009458793619712
Before: 192 After: -0.18076984080818084
Before: 238 After: 0.7405797881042117
Before: 269 After: 1.3614893206321284
Before: 275 After: 1.4816653591859188
Before: 273 After: 1.4416066796679885
Before: 152 After: -0.9819434311667831
Before: 255 After: 1.0810785640066176
Before: 105 After: -1.9233223998381406
Before: 242 After: 0.8206971471400719
Before: 236 After: 0.7005211085862816
Before: 159 After: -0.8417380528540277
Before: 225 After: 0.48019837123766596
Before: 258 After: 

In [49]:
preprocess=ColumnTransformer(transformers=[
    ("numerical",num_transformer,["satisfaction_level","last_evaluation","number_project","average_montly_hours","time_spend_company"]),
    ("categorical",ord_transformer,["salary","sales"])
])
models={
    "RandomForestClassifier":RandomForestClassifier(random_state=42),
    "SVC":SVC(random_state=42),
    "DecisionTreeClassifier":DecisionTreeClassifier(random_state=42),
    "LogisticRegression":LogisticRegression(random_state=42),
    "GaussianNB":GaussianNB()
}
classifers=Pipeline(steps=[
    ("preprocess",preprocess),
    ("classifier",RandomForestClassifier(random_state=42))
])
classifers.fit(X_train,y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  ['satisfaction_level',
                                                   'last_evaluation',
                                                   'number_project',
                                                   'average_montly_hours',
                                                   'time_spend_company']),
                                                 ('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OrdinalEncoder(categories=[['low',
                                                                                               'medium',
                                                                                               'high'],
                                                                                              array(['sales', 'accounting', 'hr', 'technical', 'support', 'management',
       'IT', 'product_mng', 'marketing', 'RandD'], dtype=object)]))]),
                                                  ['salary', 'sales'])])),
                ('classifier', RandomForestClassifier(random_state=42))])

In [50]:
y_predict=classifers.predict(X_test)
for i,j in zip(y_test,y_predict):
    print(f"Actual {i} , Prediction : {j}")

Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 1 , Prediction : 1
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 1 , Prediction : 1
Actual 1 , Prediction : 1
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 0 , Prediction : 0
Actual 0 , Prediction : 0
Actual 1 , Prediction : 1
Actual 1 , Prediction : 1
Actual 0 , P

In [51]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2294
           1       0.99      0.96      0.98       706

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.98      3000
weighted avg       0.99      0.99      0.99      3000



In [52]:
# parameters={
#     "classifier__n_estimators":[50,100,200,500],
#     "classifier__criterion":["gini","entropy","log_loss"],
#     "classifier__max_depth":[None,5,10,15,20],
#     "classifier__max_features":["sqrt","log2",None],
#     "classifier__max_leaf_nodes":[None,5,10,15,20],
# }
# model=RandomizedSearchCV(classifers,param_distributions=parameters,scoring='r2',verbose=5,n_jobs=5,cv=6,n_iter=20)
# model.fit(X_train,y_train)
# print(model.best_params_)
# print(model.best_score_)

In [53]:
accuracy = accuracy_score(y_test, y_predict)
# Giả sử y_test là nhãn thực tế và y_pred là nhãn dự đoán của mô hình
report = classification_report(y_test, y_predict)

print(report)
print(f'Accuracy: {accuracy}')

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      2294
           1       0.99      0.96      0.98       706

    accuracy                           0.99      3000
   macro avg       0.99      0.98      0.98      3000
weighted avg       0.99      0.99      0.99      3000

Accuracy: 0.9886666666666667


In [54]:
# Giả sử y_test là nhãn thực tế và y_pred là nhãn dự đoán của mô hình
cm = confusion_matrix(y_test, y_predict)

print('Confusion Matrix:')
print(cm)

Confusion Matrix:
[[2287    7]
 [  27  679]]


In [55]:
f1 = f1_score(y_test, y_predict, average='binary')  # 'binary' cho bài toán nhị phân, 'macro' cho đa lớp

print(f'F1 Score: {f1}')

F1 Score: 0.9755747126436781


In [56]:
# Huấn luyện và đánh giá từng mô hình
for name, model in models.items():
    # Tạo pipeline với tiền xử lý và mô hình
    classifer = Pipeline(steps=[
        ("preprocess", preprocess),
        ("classifier", model)
    ])

    # Huấn luyện mô hình
    classifer.fit(X_train, y_train)

    # Dự đoán và đánh giá mô hình
    y_pred = classifer.predict(X_test)

    # Tính toán độ chính xác
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model: {name}, Accuracy: {accuracy}")

Model: RandomForestClassifier, Accuracy: 0.9886666666666667
Model: SVC, Accuracy: 0.9506666666666667
Model: DecisionTreeClassifier, Accuracy: 0.9763333333333334
Model: LogisticRegression, Accuracy: 0.7636666666666667
Model: GaussianNB, Accuracy: 0.826


In [ ]:
# Định nghĩa các mô hình và các tham số để tìm kiếm (hyperparameter grid)
models = {
    "RandomForestClassifier": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "classifier__n_estimators":[50,100,200,500],
            "classifier__criterion":["gini","entropy","log_loss"],
            "classifier__max_depth":[None,5,10,15,20],
            "classifier__max_features":["sqrt","log2",None],
            "classifier__max_leaf_nodes":[None,5,10,15,20],
        }
    },
    "SVC": {
        "model": SVC(random_state=42),
        "params": {
            "classifier__C": [0.1, 1, 10],
            "classifier__kernel": ['linear', 'rbf'],
            "classifier__gamma": ['scale', 'auto']
        }
    },
    "DecisionTreeClassifier": {
        "model": DecisionTreeClassifier(random_state=42),
        "params": {
            "classifier__max_depth": [None, 5, 10, 20],
            "classifier__min_samples_split": [2, 5, 10]
        }
    },
    "LogisticRegression": {
        "model": LogisticRegression(random_state=42),
        "params": {
            "classifier__C": [0.1, 1, 10],
            "classifier__solver": ['lbfgs', 'liblinear']
        }
    },
    "GaussianNB": {
        "model": GaussianNB(),
        "params": {}  # GaussianNB không có nhiều tham số có thể tune
    },
    "XGBoost": {
        "model": XGBClassifier(random_state=42),
        "params": {
            "classifier__n_estimators": [50, 100, 200],
            "classifier__learning_rate": [0.01, 0.1, 0.3],
            "classifier__max_depth": [3, 5, 7],
            "classifier__subsample": [0.8, 1.0],
            "classifier__colsample_bytree": [0.8, 1.0]
        }
    },
    "KNeighborsClassifier": {
        "model": KNeighborsClassifier(),
        "params": {
            "classifier__n_neighbors": [3, 5, 7, 10],
            "classifier__weights": ['uniform', 'distance'],
            "classifier__metric": ['minkowski', 'euclidean', 'manhattan']
        }
    },
    "AdaBoost": {
        "model": AdaBoostClassifier(random_state=42),
        "params": {
            "classifier__n_estimators": [50, 100, 200],
            "classifier__learning_rate": [0.01, 0.1, 1],
            "classifier__algorithm": ['SAMME', 'SAMME.R']
        }
    },
}

# Lặp qua các mô hình và thực hiện fine-tuning với RandomizedSearchCV hoặc GridSearchCV
best_models = {}

for model_name, model_info in models.items():
    print(f"Training {model_name}...")

    # Tạo pipeline cho từng mô hình
    pipeline = Pipeline(steps=[
        ("preprocess", preprocess),
        ("classifier", model_info["model"])
    ])

    # Tạo RandomizedSearchCV hoặc GridSearchCV cho từng mô hình
    random_search = RandomizedSearchCV(
        pipeline,
        param_distributions=model_info["params"],
        n_iter=10,  # Chạy 10 lần tìm kiếm
        scoring='accuracy',  # Dùng độ chính xác để đánh giá
        cv=5,  # Sử dụng k-fold cross-validation
        n_jobs=-1,  # Sử dụng tất cả các lõi CPU
        verbose=5
    )

    # Huấn luyện mô hình và tìm kiếm tham số tốt nhất
    random_search.fit(X_train, y_train)

    # Lưu kết quả mô hình tốt nhất
    best_models[model_name] = random_search.best_estimator_

    # Đánh giá mô hình
    y_pred = random_search.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Best parameters for {model_name}: {random_search.best_params_}")
    print(f"Best score for {model_name}: {random_search.best_score_}")
    print(f"Accuracy: {accuracy}")

# In kết quả của mô hình tốt nhất
for model_name, model in best_models.items():
    print(f"Best model: {model_name}")
    
    # Tính toán độ chính xác
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')

    # In báo cáo phân loại chi tiết
    report = classification_report(y_test, y_pred)
    print('Classification Report:\n', report)

    # Tính toán và in ma trận nhầm lẫn
    cm = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:\n', cm)

    # Tính và in F1-score
    f1 = f1_score(y_test, y_pred, average='binary')  # Hoặc 'macro' hoặc 'weighted' cho bài toán đa lớp
    print(f'F1 Score: {f1}')
    # print(model)

Training RandomForestClassifier...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for RandomForestClassifier: {'classifier__n_estimators': 50, 'classifier__max_leaf_nodes': 20, 'classifier__max_features': None, 'classifier__max_depth': 10, 'classifier__criterion': 'entropy'}
Best score for RandomForestClassifier: 0.979914999305266
Accuracy: 0.9726666666666667
Training SVC...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for SVC: {'classifier__kernel': 'rbf', 'classifier__gamma': 'auto', 'classifier__C': 10}
Best score for SVC: 0.97108093650132
Accuracy: 0.968
Training DecisionTreeClassifier...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for DecisionTreeClassifier: {'classifier__min_samples_split': 2, 'classifier__max_depth': 10}
Best score for DecisionTreeClassifier: 0.9790819786021954
Accuracy: 0.9776666666666667
Training LogisticRegression...
Fitting 5 folds for each of 6 candidates, totalli

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for LogisticRegression: {'classifier__solver': 'liblinear', 'classifier__C': 10}
Best score for LogisticRegression: 0.7761472835903849
Accuracy: 0.7636666666666667
Training GaussianNB...
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best parameters for GaussianNB: {}
Best score for GaussianNB: 0.8288195428650826
Accuracy: 0.826
Training XGBoost...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameters for XGBoost: {'classifier__subsample': 1.0, 'classifier__n_estimators': 50, 'classifier__max_depth': 7, 'classifier__learning_rate': 0.3, 'classifier__colsample_bytree': 0.8}
Best score for XGBoost: 0.9849155550923996
Accuracy: 0.986
Training KNeighborsClassifier...
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best parameters for KNeighborsClassifier: {'classifier__weights': 'distance', 'classifier__n_neighbors': 7, 'classifier__metric': 'manhattan'}
Best score for KNeighborsClassifier: 0.9661640614144783
Accuracy: 0.967
Training AdaBoost...
Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **la

Best parameters for AdaBoost: {'classifier__n_estimators': 100, 'classifier__learning_rate': 1, 'classifier__algorithm': 'SAMME'}
Best score for AdaBoost: 0.9477458663331942
Accuracy: 0.954
Best model: RandomForestClassifier
Best model: SVC
Best model: DecisionTreeClassifier
Best model: LogisticRegression
Best model: GaussianNB
Best model: XGBoost
Best model: KNeighborsClassifier
Best model: AdaBoost


In [3]:
# import numpy as np
# from cuml import RandomForestClassifier as RandomForestClassifierGPU
# import cupy as cp
# from sklearn.ensemble import RandomForestClassifier
# import time


# # Generate some example data
# X = np.random.rand(100000, 300)  # 100000 rows, 300 columns
# y = np.random.randint(0, 2, 100000)

# # Convert data to CuPy arrays
# X_gpu = cp.array(X)
# y_gpu = cp.array(y)
# # Create and train a RandomForestClassifier on GPU
# clf = RandomForestClassifierGPU()
# start_time = time.time()
# clf.fit(X_gpu, y_gpu)
# end_time = time.time()
# # Calculate time taken for training on CPU and GPU
# time_taken_gpu = end_time - start_time
# print(f"Time taken for training on GPU: {time_taken_gpu:.2f} seconds")


Time taken for training on GPU: 1.86 seconds


In [29]:
# import cupy as cp  # CuPy để xử lý mảng trên GPU
# from cuml.ensemble import RandomForestClassifier as cuRF
# from cuml.svm import SVC as cuSVC
# from cuml.model_selection import train_test_split as cuml_train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler, OrdinalEncoder
# import pandas as pd
# import numpy as np

# # Đọc dữ liệu từ file CSV
# df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Project/HR_comma_sep.csv")
# target = "left"
# X = df.drop(labels=target, axis=1)
# y = df[target]

# # In kích thước của X và y
# print(f"X shape: {X.shape} Y shape: {y.shape}")

# # Chuyển các cột phân loại thành số
# categorical_columns = ['salary', 'sales']  # Giả sử 'salary' và 'sales' là các cột phân loại

# # Sử dụng OrdinalEncoder để chuyển đổi các cột phân loại thành số
# encoder = OrdinalEncoder()
# X[categorical_columns] = encoder.fit_transform(X[categorical_columns])

# # Chuyển dữ liệu thành CuPy arrays (xử lý trên GPU)
# X_gpu = cp.array(X.values)  # Chuyển DataFrame sang CuPy array
# y_gpu = cp.array(y.values)  # Chuyển Series sang CuPy array

# # Chia dữ liệu thành tập huấn luyện và kiểm tra sử dụng cuML (GPU-based)
# X_train, X_test, y_train, y_test = cuml_train_test_split(X_gpu, y_gpu, test_size=0.2, random_state=42)
# print(f"X train shape: {X_train.shape}, X test shape: {X_test.shape}, y train shape: {y_train.shape}, y test shape: {y_test.shape}")

# # Tiền xử lý: chuẩn hóa dữ liệu
# num_transformer = Pipeline(steps=[("scaler", StandardScaler())])

# # Định nghĩa mô hình và tham số cho các mô hình dựa trên GPU (cuML)
# models = {
#     "RandomForestClassifier (GPU)": {
#         "model": cuRF(n_estimators=100, random_state=42),
#         "params": {
#             "n_estimators": [50, 100, 200],
#             "max_depth": [None, 5, 10, 15, 20],
#             "max_features": ["sqrt", "log2", None]
#         }
#     },
#     "SVC (GPU)": {
#         "model": cuSVC(random_state=42),
#         "params": {
#             "C": [0.1, 1, 10],
#             "kernel": ['linear', 'rbf'],
#             "gamma": ['scale', 'auto']
#         }
#     }
# }

# # Khởi tạo từ điển để lưu mô hình tốt nhất
# best_models = {}

# # Lặp qua các mô hình và huấn luyện
# for model_name, model_info in models.items():
#     print(f"Đang huấn luyện {model_name}...")

#     # Tạo pipeline cho mỗi mô hình
#     pipeline = Pipeline(steps=[
#         ("preprocess", num_transformer),
#         ("classifier", model_info["model"])
#     ])

#     # Huấn luyện mô hình với dữ liệu trên GPU
#     pipeline.fit(X_train.get(), y_train.get())  # Lấy giá trị từ CuPy array và huấn luyện với dữ liệu

#     # Lưu mô hình tốt nhất tìm được từ pipeline
#     best_models[model_name] = pipeline

#     # Đánh giá mô hình tốt nhất trên tập kiểm tra
#     y_pred = pipeline.predict(X_test.get())  # Lấy giá trị từ CuPy array và dự đoán với dữ liệu
#     accuracy = accuracy_score(y_test.get(), y_pred)  # Sử dụng scikit-learn để tính accuracy

#     print(f"Độ chính xác {model_name}: {accuracy}")

# # In kết quả mô hình tốt nhất
# for model_name, model in best_models.items():
#     print(f"Mô hình tốt nhất: {model_name}")
#     # print(model)


X shape: (14999, 9) Y shape: (14999,)
X train shape: (12000, 9), X test shape: (2999, 9), y train shape: (12000,), y test shape: (2999,)
Đang huấn luyện RandomForestClassifier (GPU)...


/usr/local/lib/python3.11/dist-packages/cuml/internals/api_decorators.py:368: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return init_func(self, *args, **kwargs)


Độ chính xác RandomForestClassifier (GPU): 0.9836612204068023
Đang huấn luyện SVC (GPU)...
Độ chính xác SVC (GPU): 0.9643214404801601
Mô hình tốt nhất: RandomForestClassifier (GPU)
Pipeline(steps=[('preprocess', Pipeline(steps=[('scaler', StandardScaler())])),
                ('classifier', RandomForestClassifier())])
Mô hình tốt nhất: SVC (GPU)
Pipeline(steps=[('preprocess', Pipeline(steps=[('scaler', StandardScaler())])),
                ('classifier', SVC())])
